In [1]:
import os
import pyspark
import findspark
from pyspark.sql import SparkSession

In [2]:
try:
    findspark.init()
except:
    try:
        print(os.environ['SPARK_HOME'])
    except:
        os.environ['SPARK_HOME'] = 'C:\spark-3.0.0-preview2-bin-hadoop2.7\spark-3.0.0-preview2-bin-hadoop2.7'
        print(os.environ['SPARK_HOME'])
    try:
        print(os.environ['JAVA_HOME'])
    except KeyError:
        os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk1.8.0_231'
        print(os.environ['JAVA_HOME'])
    print(os.environ['PATH'])

In [26]:
if 'spark_data' in os.listdir():pass
else:
    import wget
    import zipfile
    wget.download('https://www.dropbox.com/sh/5c9daz43f5lstzd/AAAPN_iMkKzgqJ1k3tml7FNHa?dl=1')
    with zipfile.ZipFile('spark_data.zip', 'r') as zip_ref:
        zip_ref.extractall('spark_data')

In [3]:
spark = SparkSession.builder.master("local[*]")\
.appName('exmaple_app')\
.getOrCreate()

In [14]:
"""
스파크는 사실상 프로그래밍 언어이며 실행 계획 수립과 처리에 사용하는 자체 데이터 타입 정보를 가지고 있는 카탈리스트 엔진을 사용함.
카탈리스트 엔진은 다양한 실행 최적화 기능을 제공하며 아래 예제는 스칼라나 파이썬이 아닌 스파크의 덧셈 연산을 수행
"""

df = spark.range(500).toDF('number')
df.select(df['number']+10)

DataFrame[(number + 10): bigint]

In [22]:
"""
DataFrame과 Dataset 비교
본질적으로 구조적 API에는 '비타입형'인 DataFrame과 '타입형'인 Dataset이 있음.

DataFrame은 Row 타입으로 구성된 Dataset으로 Row 타입은 스파크가 사용하는 '연산에 최적화된 인메모리 포맷'의 내부적인 표현방식.
Row 타입을 사용하면 가비지 컬렉션과 객체 초기화 부하가 있는 JVM 데이터 타입을 사용하는 대신 자체 데이터 포맷을 사용하기 때문에 
매우 효율적인 연산이 가능

DataFrame은 컬럼과 로우로 이루어져있으며,
컬럼은 정수형이나 문자열 같은 단순 데이터 타입, 배열이나 맵 같은 복합 데이터 타입 그리고 null 값을 표현함.
로우는 데이터 레코드로 SQL, RDD, 데이터소스에서 얻거나 직접 만들 수 있음. 다음은 range 메서드를 사용해 DataFrame을 생성하는 예제
"""

spark.range(2).collect()

[Row(id=0), Row(id=1)]

In [25]:
"""
스파크는 여러가지 내부 데이터 타입을 가지고 있음
"""

from pyspark.sql.types import *

b = ByteType()
b

ByteType

In [31]:
"""
DataFrame은 Row 타입의 레코드와 각 레코드에 수행할 연산 표현식을 나타내는 여러 컬럼으로 구성됨.
스키마는 각 컬럼명과 데이터 타입을 정의하며 DataFrame의 파티셔닝은 DataFrame이나 Dataset이 클러스터에서 물리적으로 배치되는 형태를 정의함.
파티셔닝 스키마는 파티션을 배치하는 방법을 정의.

우선 DataFrame을 생성
"""

df = spark.read.format('json').load('spark_data/flight-data/json/2015-summary.json')
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [38]:
"""
스키마는 DataFrame의 컬럼명과 데이터 타입을 정의함. 데이터소스에서 스키마를 얻거나 직접 정의할 수 있음.

이번 예제에서는 앞서 사용한 데이터를 사용. 이 파일은 미국 교통통계국이 제공하는 항공운항 데이터이며 줄로 구분된 반정형 json 데이터임
스키마는 여러 개의 StructField 타입 필드로 구성된 StructType 객체이며 이름, 데이터 타입, 컬럼이 값이 없거나 null일 수 있는지
지정하는 불리언값을 가짐.
"""

spark.read.format('json').load('spark_data/flight-data/json/2015-summary.json').schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [47]:
"""
스키마는 복합 데이터타입인 StructType을 가질수있음. 
스파크는 런타임에 데이터 타입이 스키마의 데이터 타입과 일치하지 않으면 오류를 발생시킴

metadata는 뭔지 몰겠음...
"""
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField('DEST_COUNTRY_NAME', StringType(), True),
    StructField('ORIGIN_COUNTRY_NAME', StringType(), True),
    StructField('count', LongType(), False,metadata = {'hello':'world'})
])

df = spark.read.format('json').schema(myManualSchema).load('spark_data/flight-data/json/2015-summary.json')
df.head(10)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))